In [93]:
import os, json, time, logging
from datetime import datetime
import numpy as np

from scipy.optimize import minimize

from qiskit.quantum_info import SparsePauliOp

from qiskit_aer import Aer, AerSimulator
from qiskit_aer.primitives import EstimatorV2 as AerEstimator
from qiskit_aer.noise import NoiseModel

from qiskit_ibm_runtime import EstimatorV2 as Estimator, QiskitRuntimeService, Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import StatevectorEstimator

from susy_qm import calculate_Hamiltonian, ansatze

from qiskit.circuit import QuantumCircuit, Parameter

In [94]:
path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\apikey.json"
with open(path, encoding="utf-8") as f:
    api_key = json.load(f).get("apikey")

NQCC_IBM_QUANTUM_API_KEY = api_key
ibm_instance_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/d4f95db0515b47b7ba61dba8a424f873:55736fd5-c0a0-4f44-8180-ce6e81d6c9d0::"
service = QiskitRuntimeService(channel="ibm_quantum_platform", token=NQCC_IBM_QUANTUM_API_KEY, instance=ibm_instance_crn)

In [95]:
potential = "DW"
cutoff = 4

ansatze_type = 'exact'
if potential == "QHO":
    ansatz_name = f"CQAVQE_QHO_{ansatze_type}"
elif (potential != "QHO") and (cutoff <= 16):
    ansatz_name = f"CQAVQE_{potential}{cutoff}_{ansatze_type}"
else:
    ansatz_name = f"CQAVQE_{potential}16_{ansatze_type}"

ansatz = ansatze.get(ansatz_name)
num_params = ansatz.n_params

print(f"Running VQE with {ansatz_name} ansatz using {num_params} params")

Running VQE with CQAVQE_DW4_exact ansatz using 3 params


In [96]:
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eigvals(H))[:4]
num_qubits = int(1 + np.log2(cutoff))

In [97]:
observable = SparsePauliOp.from_operator(H)
print(observable)

SparsePauliOp(['III', 'IIX', 'IXI', 'IXX', 'IXZ', 'IYY', 'IZI', 'IZX', 'IZZ', 'ZII', 'ZIX', 'ZXX', 'ZYY', 'ZZX'],
              coeffs=[ 5.375     +0.j,  3.027187  +0.j,  2.41481457+0.j,  2.4330127 +0.j,
 -0.64704761+0.j,  1.5669873 -0.j, -1.75      +0.j, -1.25942005+0.j,
 -1.75      +0.j,  0.5       +0.j,  0.96592583+0.j,  0.5       +0.j,
  0.5       -0.j, -0.25881905+0.j])


In [98]:
#param_objs = [Parameter(f"θ{i}") for i in range(num_params)]
#qc = QuantumCircuit(num_qubits)
#n = num_qubits-1
#qc.x(n)
#qc.ry(param_objs[0], n)

qc = ansatze.pl_to_qiskit(ansatz, num_qubits=num_qubits, reverse_bits=True)
print(qc)  

     ┌────────┐┌────────┐
q_0: ┤ Ry(θ1) ├┤ Ry(θ2) ├
     ├────────┤└───┬────┘
q_1: ┤ Ry(θ0) ├────■─────
     └─┬───┬──┘          
q_2: ──┤ X ├─────────────
       └───┘             


In [99]:
params = [
            0.3966509377256218,
            5.951916764806598,
            6.008466886069231
        ]

estimator = StatevectorEstimator()
pub = (qc, observable, params)
result = estimator.run(pubs=[pub]).result()
energy = result[0].data.evs
print(energy)

0.9065598714743937


In [100]:
num_params = 3
param_objs = [Parameter(f"θ{i}") for i in range(num_params)]

qc = QuantumCircuit(num_qubits)
qc.x(2)
qc.ry(param_objs[0], 0)
qc.ry(param_objs[1], 1)
qc.cry(param_objs[2], 1,0)
print(qc)

params = [  5.951916764806598,
            0.3966509377256218,
            6.008466886069231
        ]

estimator = StatevectorEstimator()
pub = (qc, observable, params)
result = estimator.run(pubs=[pub]).result()
energy = result[0].data.evs
print(energy)

     ┌────────┐┌────────┐
q_0: ┤ Ry(θ0) ├┤ Ry(θ2) ├
     ├────────┤└───┬────┘
q_1: ┤ Ry(θ1) ├────■─────
     └─┬───┬──┘          
q_2: ──┤ X ├─────────────
       └───┘             
0.9065598714743937


In [101]:
num_params = 3
param_objs = [Parameter(f"θ{i}") for i in range(num_params)]

qc = QuantumCircuit(num_qubits)
qc.x(2)
qc.ry(param_objs[0], 0)
qc.ry(param_objs[1], 1)
qc.cry(param_objs[2], 1,0)
print(qc)

params = [  5.951916764806598,
            0.3966509377256218,
            6.008466886069231
        ]

backend = AerSimulator(method='statevector')
estimator = Estimator(mode=backend)
estimator.options.default_shots = 100000

pub = (qc, observable, params)
result = estimator.run(pubs=[pub]).result()
energy = result[0].data.evs
print(energy)

     ┌────────┐┌────────┐
q_0: ┤ Ry(θ0) ├┤ Ry(θ2) ├
     ├────────┤└───┬────┘
q_1: ┤ Ry(θ1) ├────■─────
     └─┬───┬──┘          
q_2: ──┤ X ├─────────────
       └───┘             
0.899618195785681


In [9]:
#noise_model = NoiseModel.from_backend('ibm_kingston')
#backend = AerSimulator(noise_model=noise_model)
backend = service.backend('ibm_kingston')
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
ansatz_isa = pm.run(qc)

In [11]:
print(ansatz_isa)

          ┌────────┐┌────┐┌────────┐┌────────────┐┌────┐┌────────┐
q_1 -> 10 ┤ Rz(-π) ├┤ √X ├┤ Rz(-π) ├┤ Rz(π + θ0) ├┤ √X ├┤ Rz(3π) ├
          └────────┘└────┘└────────┘└────────────┘└────┘└────────┘


In [12]:
layout = getattr(ansatz_isa, "layout", None)
hamiltonian_isa = observable.apply_layout(layout) if layout else observable

print(hamiltonian_isa)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII'],
              coeffs=[0.5+0.j, 0.5+0.j])
